<a href="https://colab.research.google.com/github/bhanuchaddha/The-Ai-Handbook/blob/main/4-ai-agents/Multi_Agents_Article_Reseracher_and_Write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [16]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [2]:
from crewai import Agent, Task, Crew

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

# Creating the Crew
Create your crew of Agents
Pass the tasks to be performed by those agents.

**Note**: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.

`verbose=2` allows you to see all the logs of the execution.

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [17]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
# Comprehensive Content Plan Document on Artificial Intelligence

## 1. Latest Trends in Artificial Intelligence
- **Generative AI**: The rise of AI tools capable of generating content, including text, images, and even music.
- **Explainable AI (XAI)**: Increased focus on making AI decisions interpretable and transparent to users.
- **AI in Healthcare**: Growing use of AI for diagnostics, personalized medicine, and managing patient care.
- **AI Ethics and Regulations**: Ongoing discussions and initiatives aimed at establishing ethical guidelines and regulatory practices for AI development 

In [19]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# The Dynamic Frontier of Artificial Intelligence in 2023

## Introduction

Artificial Intelligence (AI) has rapidly evolved from a futuristic concept to a daily reality in modern society. It permeates various facets of our lives, from the systems powering our smartphones to the algorithms that recommend our next binge-watch series. In 2023, AI stands at a crossroads, characterized by exciting advancements that promise to reshape industries, enhance personal experiences, and provoke necessary discussions about ethics and regulations. This article delves into the latest trends, key players, and recent innovations in the AI landscape, providing insights that showcase both the potential and challenges of this extraordinary technology.

As we explore the current state of AI, we will examine the emergence of generative AI tools, the importance of explainable AI, the profound impact of AI in healthcare, and the ongoing debates surrounding AI ethics and regulations. Each of these aspects not only reflects how far we've come but also serves as a harbinger of what lies ahead, inviting both enthusiasm and scrutiny.

## Current Trends in AI

### Generative AI: The Creative Frontier

One of the most notable trends in AI today is the rise of generative AI — powerful tools capable of producing a wide array of content, including text, images, and music. These systems leverage advanced machine learning techniques and vast datasets to create original, contextually relevant outputs. For instance, OpenAI’s GPT-3 has demonstrated remarkable capabilities in generating articles, poetry, and even programming code, while DALL-E can create strikingly realistic images from textual descriptions. As generative AI becomes more accessible, it opens up new creative avenues for artists, writers, and businesses alike.

Moreover, the implications of generative AI extend beyond creativity. Businesses are harnessing these tools to enhance customer interaction through personalized marketing content and streamlined communications. This bridging of automation and genuine human engagement is transforming how companies interact with their audiences and can lead to significant increases in efficiency and customer satisfaction.

### Explainable AI: Building Trust Through Transparency

As AI systems become more ingrained in decision-making processes, the need for explainability surfaces. Explainable AI (XAI) focuses on making AI decisions interpretable and transparent to users. By providing insights into how AI reaches conclusions, stakeholders can better trust these systems—this is particularly crucial in sensitive fields like healthcare and finance. XAI fosters accountability, allowing for the necessary scrutiny that can ultimately improve outcomes for all involved parties.

For example, AI-driven diagnostics in healthcare must not only deliver accurate results but also explain the reasoning behind those results to clinicians. This builds trust and equips healthcare professionals with the knowledge needed to make informed decisions for their patients. Enhanced transparency in AI systems promotes better communication and understanding between technology and its users.

### The Role of AI in Healthcare

AI's transformative power is profoundly felt in the healthcare sector, where it plays an increasingly pivotal role in diagnostics, personalized medicine, and patient care management. Tools powered by AI analyze medical images, predict patient outcomes, and suggest tailored treatment options based on unique genetic profiles. This has not only enhanced the speed of diagnoses but also significantly improved accuracy, ultimately leading to better patient outcomes.

Numerous case studies highlight the success of AI implementations in major hospital systems, proving that these technologies can expedite processes and reduce human error. As the age of personalized medicine dawns, the integration of AI is expected to refine how healthcare providers approach treatment—shifting from a one-size-fits-all model to a personalized approach tailored to individual patients.

## Key Players in the AI Landscape

### Industry Giants Leading the Charge

Several key players are shaping the future of AI through innovation and investment. OpenAI remains at the forefront with its advanced language models such as GPT-3 and image generation via DALL-E. These tools are continuously evolving and contributing to the wider landscape of AI technologies, pushing the limits of what is possible.

Google DeepMind has made significant strides in machine learning research, spearheading initiatives that focus on solving complex challenges by implementing AI in innovative ways. IBM Watson emphasizes practical applications in healthcare and customer service, tailoring AI solutions to address specific industry needs. Each player's contribution helps build an expansive knowledge base and a competitive landscape in AI.

### Collaborations and Innovations

The partnerships among these key players are fostering collaborative ecosystems that propel AI innovations. Notably, the Microsoft and OpenAI partnership is expanding to incorporate sophisticated AI models into Microsoft's products, enriching user experiences and streamlining workflows across business applications. Such strategic collaborations not only enhance the offerings from these giants but also signal an industry-wide trend toward integrating AI solutions for greater efficiency and effectiveness.

Additionally, these collaborations pave the way for groundbreaking innovations that can alter how industries operate. By sharing expertise and resources, these organizations can address challenges more effectively and accelerate the development and adoption of AI technologies across various fields.

## Recent Innovations and News

### New Regulatory Landscapes

As AI continues to permeate daily life, new regulations are emerging to ensure safety and ethical deployment. The European Union's recent proposed regulations are a response to growing concerns about the integrity and accountability of AI systems. These regulations aim to establish frameworks that prioritize ethical considerations and user protection while fostering innovation within the AI sector.

The implications of such regulations may facilitate a more robust understanding of AI technologies among the general public, while simultaneously holding organizations accountable for responsible AI usage. As these discussions unfold, the balance between fostering innovation and ensuring ethical practices becomes paramount in the development of AI.

### AI in Cybersecurity

In the face of increasing cyber threats, AI is rapidly becoming an indispensable ally for cybersecurity professionals. The deployment of AI tools allows organizations to predict, detect, and neutralize threats with unprecedented efficiency. AI can analyze vast datasets for unusual patterns indicative of potential breaches, thus enabling more proactive responses to cyber incidents.

With the increasing sophistication of cybercriminals, the advancement of AI in cybersecurity is not merely an option but a necessity. As awareness of AI's capabilities in this sector gains traction, organizations are recognizing the potential to fortify their defenses by leveraging these cutting-edge technologies.

## Conclusion

The landscape of Artificial Intelligence continues to evolve dramatically, characterized by transformative trends, influential players, and groundbreaking innovations. As AI technologies such as generative AI and explainable AI redefine expectations across various industries, their implications become more pervasive and significant. The contributions of organizations like OpenAI, Google DeepMind, and IBM Watson further illuminate the interconnectedness of creativity, healthcare, and machine learning.

As we look to the future, it is essential for individuals and businesses alike to contemplate the ethical dimensions of AI. The potential for both remarkable advancements and significant challenges underscores the importance of developing a framework that promotes responsible and inclusive AI usage. Ultimately, the choices we make today regarding AI will shape the society of tomorrow.

## Call to Action

We encourage our readers to stay informed about the latest developments in AI by subscribing to our updates. Share your thoughts on these advancements in the comments below and explore our related articles to deepen your understanding of how AI continues to impact our lives and industries.
```

In [20]:
topic = "LLMs"
result = crew.kickoff(inputs={"topic": topic})
Markdown(result.raw)

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on LLMs.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan Document: LLMs (Large Language Models)**

**1. Latest Trends in LLMs:**
   - **Advancements in Model Architectures:** The shift towards more efficient and smaller models (e.g., distillation techniques).
   - **Proliferation of Open-Source Models:** An increase in the availability of LLMs on platforms like Hugging Face, encouraging community involvement.
   - **Focus on Multimodal Capabilities:** Emerging models that combine text with images and audio (e.g., OpenAI’s CLIP, Google's PaLM).
   - **Ethical and Responsible AI:** Heightened discussions around bias in models and the establishment of 

```markdown
# Understanding Large Language Models: Trends, Players, and Insights

## Introduction
In recent years, Large Language Models (LLMs) have rapidly risen to prominence, revolutionizing how we interact with technology. These sophisticated algorithms, capable of understanding and generating human-like text, serve as the backbone for a range of applications—from chatbots to advanced decision-making tools in various industries. Their significance extends not just to technological advancements but also to ethical considerations, consumer safety, and the democratization of information.

As we move further into the era of artificial intelligence, it becomes crucial to understand these models—not only to harness their capabilities for innovation but also to navigate the ethical complexities they present. In this blog, we will explore the latest trends in LLMs, introduce key players in the space, highlight noteworthy news developments, and discuss practical applications, providing you with a comprehensive overview of this rapidly evolving landscape.

## Latest Trends in LLMs
### Advancements in Model Architectures
One of the most exciting trends in the world of LLMs is the shift towards more efficient and smaller model architectures. Techniques such as model distillation are gaining traction, allowing large models to be compressed without sacrificing performance. This trend not only speeds up processing times but also reduces the computational power needed, making LLMs more accessible for smaller organizations and individuals. 

Moreover, the evolution of architectures can significantly enhance the user experience by enabling quicker, more responsive interactions. As efficiency continues to improve, we can expect a broader adoption of LLM technologies across various sectors—creating opportunities for innovative applications that were previously unfeasible.

### The Rise of Multimodal Capabilities
Another captivating area within LLMs is the development of multimodal capabilities, which combine text with other forms of data, such as images and audio. Models like OpenAI's CLIP and Google’s PaLM are pioneers in this field, offering users the ability to interact with AI in more dynamic and meaningful ways. This evolution signifies a transition from text-only models to more versatile systems that can cater to a variety of user interactions. 

The integration of multiple data formats not only enriches the user experience but also broadens the potential applications of LLMs, impacting fields like education, marketing, and healthcare. With these advancements, we are witnessing a transformation that enhances how information is processed and communicated, fostering richer interactions between humans and machines.

### Ethical and Responsible AI
With great power comes great responsibility. The dialogue surrounding ethical and responsible AI is becoming increasingly vital, especially as bias in models becomes more apparent. Organizations are establishing ethical guidelines for LLM use, promoting fairness, transparency, and accountability. It’s essential for researchers and developers to address these concerns through ongoing training and evaluation, ensuring LLMs benefit society without perpetuating harmful biases.

Additionally, as LLM technology continues to evolve, engaging with diverse perspectives will be crucial to forming a holistic understanding of its implications. Continuous collaboration among technologists, ethicists, and policymakers can provide a framework to navigate the complexities of AI responsibly while fostering innovation that positively impacts users.

## Key Players in the LLM Space
### Leading Innovators
Several key players are significantly shaping the LLM landscape. OpenAI, renowned for its development of ChatGPT, continues to push the frontiers of AI research with innovative approaches that redefine the ways we engage with technology. Google DeepMind closely follows, contributing breakthroughs with models like BERT and PaLM that reshape natural language understanding and facilitate a more nuanced interaction with AI.

As these companies and others advance their research, the competitive landscape fuels remarkable progress, ensuring that LLMs remain at the forefront of technological development. Staying aware of these players helps consumers and businesses alike to identify tools and technologies that best align with their needs.

### The Open-Source Movement
Open-source platforms like Hugging Face play a crucial role in democratizing access to LLMs, allowing developers and researchers to collaborate and innovate freely. This availability encourages community involvement and accelerates innovation, making it easier for organizations to deploy LLM technology and expand its applications across various sectors, from finance to healthcare.

By fostering an open-source environment, we empower a wider range of contributors to enhance and refine LLM technologies. This approach not only stimulates growth and diversity in the field but also promotes a collaborative spirit that can lead to groundbreaking advancements, ensuring that LLM development is a collective endeavor.

## Current News and Updates in LLMs
### Major Announcements
The LLM market is buzzing with recent developments, such as OpenAI's updates to ChatGPT, which enhance user experience with new plugins and API access. Meanwhile, Google's introduction of Bard promises to refine search engine optimization through advanced language processing capabilities, offering users a more personalized and effective search experience.

These updates signal a commitment to improving AI interactions and enhancing the capabilities of LLMs, making them more functional and user-friendly. As new features are rolled out, users can expect to benefit from a more robust and versatile toolkit for engaging with LLM technologies.

### Regulatory Challenges and Implications
As the capabilities of LLMs expand, so does the need for regulation. Discussions in Europe and the United States regarding the ethical implications of AI usage are intensifying. Policymakers are focused on creating frameworks that ensure the responsible deployment of LLMs while mitigating risks associated with bias, misinformation, and data privacy.

The establishment of regulatory guidelines will play a vital role in shaping the future landscape of LLMs. As organizations grow increasingly reliant on these technologies, fostering a well-defined legal environment will help ensure that advances in AI serve the public interest while maintaining accountability and ethical standards.

## Practical Applications of LLMs
### Real-World Utilizations
LLMs are now being integrated into various workflows to improve efficiency and decision-making processes. For instance, healthcare providers use these models to analyze patient data and generate insights for personalized treatment plans. In finance, LLMs help develop chatbots that provide tailored financial advice, enabling organizations to enhance customer service while streamlining operations.

These practical applications underscore the transformative potential of LLM technology, which is set to shape how industries operate and cater to their stakeholders. From facilitating better healthcare outcomes to redefining customer engagement, LLMs are paving the way for innovation across multiple sectors.

### Case Studies Demonstrating Impact
A notable case study involves educators leveraging LLMs to create personalized learning materials for students. By analyzing individual learning styles and performance data, these models can generate customized content to meet specific needs, thus enhancing the educational experience. Such implementations showcase the impactful nature of LLM technology in real-world settings.

As LLMs continue to evolve and be applied across various fields, the potential for positive change is immense. These case studies highlight how AI can be responsibly harnessed to improve lives, foster creativity, and enhance productivity, demonstrating the true value of LLMs in contemporary society.

## Conclusion
Understanding and leveraging Large Language Models is increasingly essential in a technology-driven world. They provide remarkable opportunities for innovation while posing significant ethical challenges that require thoughtful solutions. As these models become integral in supporting various industries, organizations and individuals must stay informed to utilize them responsibly and effectively.

We encourage readers to engage with the content, share their experiences with LLMs, or explore practical applications in their fields. Subscribe for the latest updates, participate in webinars on LLM implementation, and join community forums to deepen your understanding of this revolutionary technology.
```